In [9]:
import ee
import geemap
import pandas as pd
import numpy as np

In [ ]:
def get_brightness(image):
    """
    Compute the brightness of an image by summing the visible bands (B2, B3, B4).
    You can also include infrared bands (e.g., B5, B6) if you want.
    """
    brightness = image.select(['B2', 'B3', 'B4']).reduce(ee.Reducer.sum())
    return brightness

In [ ]:
def select_brightest_pixels(image, n=10):
    """
    Select the n brightest pixels from the image based on brightness.
    """
    # Compute brightness for each pixel
    brightness = get_brightness(image)

    # Sample the image to get all pixels in the region of interest (ROI)
    sampled_points = brightness.sample(region=pointed, scale=30, numPixels=1000)

    # Sort the points by brightness in descending order
    sorted_points = sampled_points.sort('sum', False)

    # Take the top n brightest points
    top_n_brightest_points = sorted_points.limit(n)

    return top_n_brightest_points

In [ ]:
def get_average_of_brightest(image, n=10):
    """
    For the top n brightest pixels, calculate the mean value for each band.
    """
    brightest_points = select_brightest_pixels(image, n)
    
    # Calculate the mean for each band (e.g., B2, B3, B4, etc.)
    means = brightest_points.reduceColumns(ee.Reducer.mean(), ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8']).get('mean')
    return ee.Dictionary(means)

In [ ]:
def spectral_unmixing(image, n=10):
    """
    Perform spectral unmixing using the average values of the top n brightest pixels as endmembers.
    """
    # Get the average endmember values from the brightest pixels for each image
    average_endmembers = get_average_of_brightest(image, n)
    
    # Convert the dictionary of mean values into a list (required format for unmixing)
    endmembers = [
        ee.List([average_endmembers.get('B2'), average_endmembers.get('B3'), average_endmembers.get('B4')]),
        ee.List([average_endmembers.get('B5'), average_endmembers.get('B6'), average_endmembers.get('B7')]),
    ]
    
    # Perform unmixing using the average endmembers
    unmixed_image = image.unmix(endmembers, True, True)
    
    return unmixed_image.set('system:time_start', image.get('system:time_start'))

In [ ]:
def process_images():
    image_list = l8_clipped_forexport.toList(l8_clipped.size().getInfo())
    num_images = image_list.size().getInfo()
    
    for i in range(num_images):
        image = ee.Image(image_list.get(i))
        
        # Apply the spectral unmixing using the average of the 10 brightest pixels
        unmixed_image = spectral_unmixing(image)
        
        # Optionally export the unmixed image or process it further
        # export_image(unmixed_image)  # Uncomment this line to export unmixed images
        print(f"Spectral unmixing completed for image with date {image.date().format('YYYY-MM-dd').getInfo()}.")

In [ ]:
image_viz_params = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 0.5,
    'gamma': [0.95, 1.1, 1],
}

Map = geemap.Map(zoom=10, center=[-77.616808, 163.077952])
Map.addLayer(unmixed_example, image_viz_params, 'Unmixed Example')
Map.addLayer(lake1, image_viz_params, 'Original RGB')
Map.addLayer(roi_ice)
Map